*2024/11/05*
***
# 线性神经网络
## softmax回归
分类问题-概率

### 1 分类问题
从一个图像分类问题开始。假设每次输入是一个$2\times2$的灰度图像。我们可以用一个标量表示每个像素值，每个图像对应四个特征$x_1, x_2, x_3, x_4$。此外，假设每个图像属于类别“猫”“鸡”和“狗”中的一个。

接下来，我们要选择如何表示标签。我们有两个明显的选择：最直接的想法是选择$y \in \{1, 2, 3\}$，其中整数分别代表$\{\text{狗}, \text{猫}, \text{鸡}\}$。这是在计算机上存储此类信息的有效方法。如果类别间有一些自然顺序，比如说我们试图预测$\{\text{婴儿}, \text{儿童}, \text{青少年}, \text{青年人}, \text{中年人}, \text{老年人}\}$，那么将这个问题转变为回归问题，并且保留这种格式是有意义的。

但是一般的分类问题并不与类别之间的自然顺序有关。幸运的是，统计学家很早以前就发明了一种表示分类数据的简单方法：*独热编码*（one-hot encoding）。独热编码是一个向量，它的分量和类别一样多。类别对应的分量设置为1，其他所有分量设置为0。在我们的例子中，标签$y$将是一个三维向量，其中$(1, 0, 0)$对应于“猫”、$(0, 1, 0)$对应于“鸡”、$(0, 0, 1)$对应于“狗”：

$$y \in \{(1, 0, 0), (0, 1, 0), (0, 0, 1)\}.$$

***
**独热编码:**
特征数字化：将分类变量（或称为离散特征、无序特征）转换为一种适合机器学习算法处理的格式。

为每个分类特征的每个可能值创建一个新的二进制特征（即“独热”特征），其中只有一个特征在任何给定时间被激活（标记为1），而其他所有特征都被标记为0。

独热编码（One-Hot Encoding）：使用N位状态寄存器对N个状态进行编码，每个状态由其独立的寄存器位表示，并且任意时刻只有一位是有效的（即设置为1）。

![image.png](https://i-blog.csdnimg.cn/blog_migrate/d5ded34520d72e6b35822ebde30aac86.jpeg)

- 优点：
    - 解决分类数据处理问题：独热编码将离散分类特征转换为机器学习算法易于处理的二进制格式，提高了算法对离散特征的处理能力。

    - 避免引入数值偏误：通过将每个类别映射到独立的二进制向量，独热编码消除了类别间可能存在的错误数值关系，从而避免了算法基于这些关系做出不准确的预测。

- 缺点：

    - 维度增加：当类别数量较多时，独热编码会显著增加特征空间的维度，可能导致计算复杂性和过拟合问题。

    - 信息损失风险：独热编码可能无法充分捕捉类别间的潜在关系或顺序信息，从而在某些情况下导致有用信息的丢失。
***

### 2 网络架构
为了估计所有可能类别的条件概率，我们需要一个有多个输出的模型，每个类别对应一个输出。为了解决线性模型的分类问题，我们需要和输出一样多的*仿射函数*（affine function）。每个输出对应于它自己的仿射函数。在我们的例子中，由于我们有4个特征和3个可能的输出类别，我们将需要12个标量来表示权重（带下标的$w$），3个标量来表示偏置（带下标的$b$）。下面我们为每个输入计算三个*未规范化的预测*（logit）：$o_1$、$o_2$和$o_3$。

$$
\begin{aligned}
o_1 &= x_1 w_{11} + x_2 w_{12} + x_3 w_{13} + x_4 w_{14} + b_1,\\
o_2 &= x_1 w_{21} + x_2 w_{22} + x_3 w_{23} + x_4 w_{24} + b_2,\\
o_3 &= x_1 w_{31} + x_2 w_{32} + x_3 w_{33} + x_4 w_{34} + b_3.
\end{aligned}
$$

我们可以用神经网络图来描述这个计算过程。与线性回归一样，softmax回归也是一个单层神经网络。由于计算每个输出$o_1$、$o_2$和$o_3$取决于所有输入$x_1$、$x_2$、$x_3$和$x_4$，所以softmax回归的输出层也是全连接层。

![softmax回归是一种单层神经网络](./img/softmaxreg.svg)

为了更简洁地表达模型，我们仍然使用线性代数符号。通过向量形式表达为$\mathbf{o} = \mathbf{W} \mathbf{x} + \mathbf{b}$，这是一种更适合数学和编写代码的形式。由此，我们已经将所有权重放到一个$3 \times 4$矩阵中。
对于给定数据样本的特征$\mathbf{x}$，我们的输出是由权重与输入特征进行矩阵-向量乘法再加上偏置$\mathbf{b}$得到的。

### 3 参数开销
对于任何具有$d$个输入和$q$个输出的全连接层，参数开销为$\mathcal{O}(dq)$。

### 4 softmax运算
希望模型的输出$\hat{y}_j$可以视为属于类$j$的概率，然后选择具有最大输出值的类别$\operatorname*{argmax}_j y_j$作为我们的预测。例如，如果$\hat{y}_1$、$\hat{y}_2$和$\hat{y}_3$分别为0.1、0.8和0.1，那么我们预测的类别是2，在我们的例子中代表“鸡”。

然而不能直接将未规范化的预测$o$直接视作输出。因为将线性层的输出直接视为概率时存在一些问题：一方面，我们没有限制这些输出数字的总和为1。另一方面，根据输入的不同，它们可以为负值。这些违反了概率基本公理。

因此，需要一个训练的目标函数，来激励模型精准地估计概率。

例如，在分类器输出0.5的所有样本中，我们希望这些样本是刚好有一半实际上属于预测的类别。这个属性叫做*校准*（calibration）。

社会科学家邓肯·卢斯于1959年在*选择模型*（choice model）的理论基础上发明了*softmax函数*。softmax函数能够将未规范化的预测变换为非负数并且总和为1，同时让模型保持可导的性质。

为了完成这一目标，我们首先对每个未规范化的预测求幂，这样可以确保输出非负。为了确保最终输出的概率值总和为1，我们再让每个求幂后的结果除以它们的总和。如下式：

$$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o})\quad \text{其中}\quad \hat{y}_j = \frac{\exp(o_j)}{\sum_k \exp(o_k)}$$






